<a href="https://colab.research.google.com/github/dorobat-diana/LicentaAi/blob/main/SiameseFinetunning_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from google.colab import drive

# --- 0. Mount Google Drive ---
try:
    drive.mount('/content/drive')
except:
    print("Google Drive already mounted or mount failed.")

# --- 1. Configuration & Paths ---
# Path to the Siamese model trained in the FIRST phase
TRAINED_SIAMESE_MODEL_PATH = '/content/drive/MyDrive/ColabNotebooks/results/siamese_mobilenetv2_landmark_model_v2.keras'
# Path to save the model after this SECOND fine-tuning phase
FINETUNED_SIAMESE_MODEL_PATH = '/content/drive/MyDrive/ColabNotebooks/results/siamese_mobilenetv2_landmark_model_v2_top_layers_finetuned.keras' # Changed name

TRAIN_DIR = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/split/train'
TEST_DIR = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/split/test' # For validation pairs

IMG_WIDTH, IMG_HEIGHT = 224, 224
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
BATCH_SIZE = 16 # Consider reducing batch size if memory becomes an issue with more trainable params
EPOCHS_FINETUNE = 25 # Adjust as needed, use EarlyStopping
LEARNING_RATE_FINETUNE = 1e-5 # CRITICAL: Use a very low learning rate
N_TOP_LAYERS_TO_UNFREEZE = 5 # Number of layers from the END of MobileNetV2 to unfreeze. Adjust this!
                               # For MobileNetV2, a block can have multiple layers (Conv, BN, ReLU, DepthwiseConv etc.)
                               # For example, unfreezing from 'block_13_expand' onwards.
                               # Inspect your feature_extractor_layer_in_siamese.summary() to see layer names and count.

# --- 2. Load Pre-Trained Siamese Model ---
print("Loading the previously trained Siamese model...")
try:
    tf.keras.config.enable_unsafe_deserialization()
    siamese_model = load_model(TRAINED_SIAMESE_MODEL_PATH, compile=False)
    print("Trained Siamese model loaded successfully.")
    # Assuming LEARNING_RATE_FINETUNE is defined as in your script
    siamese_model.compile(
      loss='binary_crossentropy',
      optimizer=Adam(learning_rate=LEARNING_RATE_FINETUNE),
      metrics=['accuracy']
    )
    print("Original Siamese model summary:")
    siamese_model.summary()
except Exception as e:
    print(f"Error loading the Siamese model: {e}")
    raise

# --- 3. Unfreeze TOP Layers in the Feature Extractor part of the Siamese Model ---
print("\nAttempting to unfreeze TOP layers of the feature extractor...")
try:
    feature_extractor_layer_in_siamese = siamese_model.get_layer("feature_extractor_functional")
    print(f"Found feature extractor layer: {feature_extractor_layer_in_siamese.name} of type {type(feature_extractor_layer_in_siamese)}")

    # First, set the entire sub-model (feature_extractor) to non-trainable,
    # then selectively unfreeze its top layers. This ensures only the intended layers are unfrozen.
    feature_extractor_layer_in_siamese.trainable = False
    print(f"Set '{feature_extractor_layer_in_siamese.name}' to non-trainable initially.")

    # Check if this layer is a Keras Model itself, which should have a 'layers' attribute
    if hasattr(feature_extractor_layer_in_siamese, 'layers') and isinstance(feature_extractor_layer_in_siamese, Model):
        print(f"'{feature_extractor_layer_in_siamese.name}' is a Keras Model. Unfreezing its top {N_TOP_LAYERS_TO_UNFREEZE} layers.")

        # It's good practice to keep BatchNormalization layers frozen when fine-tuning,
        # especially if the new dataset/batch size is small, to use their pre-trained statistics.
        # However, some argue that for end-to-end fine-tuning, they should also adapt.
        # Let's try keeping them frozen first.
        unfrozen_count = 0
        for layer in feature_extractor_layer_in_siamese.layers[-N_TOP_LAYERS_TO_UNFREEZE:]:
            # Crucial: Do NOT make Batch Normalization layers trainable when fine-tuning with small batch sizes
            # or if you want to preserve the statistics learned from the larger pre-training dataset.
            # MobileNetV2 heavily relies on Batch Norm.
            if not isinstance(layer, BatchNormalization):
                layer.trainable = True
                unfrozen_count += 1
                # print(f"  Layer '{layer.name}' is now TRAINABLE.")
            # else:
                # print(f"  Layer '{layer.name}' (BatchNormalization) remains FROZEN.")
        print(f"Successfully unfroze {unfrozen_count} non-BatchNormalization layers from the top {N_TOP_LAYERS_TO_UNFREEZE} of '{feature_extractor_layer_in_siamese.name}'.")
        if unfrozen_count == 0 and N_TOP_LAYERS_TO_UNFREEZE > 0:
            print(f"WARNING: No layers were unfrozen. Check N_TOP_LAYERS_TO_UNFREEZE ({N_TOP_LAYERS_TO_UNFREEZE}) or if all top layers are BatchNormalization.")

        # If you want to unfreeze Batch Norm layers as well (use with caution):
        # for layer in feature_extractor_layer_in_siamese.layers[-N_TOP_LAYERS_TO_UNFREEZE:]:
        #     layer.trainable = True
        # print(f"Unfroze all top {N_TOP_LAYERS_TO_UNFREEZE} layers of '{feature_extractor_layer_in_siamese.name}' (including BN).")


    else:
        print(f"Warning: Layer '{feature_extractor_layer_in_siamese.name}' is not a Keras Model or has no 'layers' attribute. Cannot partially unfreeze its internal layers this way.")
        print("If this is unexpected, ensure 'feature_extractor_functional' was indeed a tf.keras.Model.")
        print("Setting the entire feature_extractor_layer_in_siamese to trainable as a fallback (original Option 1 behavior).")
        feature_extractor_layer_in_siamese.trainable = True


    print("\nSiamese model summary AFTER unfreezing feature extractor part:")
    siamese_model.summary()

except Exception as e:
    print(f"Error during unfreezing process: {e}")
    print("Ensure the feature extractor model name 'feature_extractor_functional' matches how it was named and used in the first phase,")
    print("and that it is a tf.keras.Model instance.")
    raise


# --- 4. Prepare Data (with Augmentation) ---

def augment_image(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.15)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image

def load_and_preprocess_image_tf_augmented(path_tensor, img_shape=(IMG_WIDTH, IMG_HEIGHT, 3), augment=False):
    img = tf.io.read_file(path_tensor)
    try:
        img = tf.image.decode_image(img, channels=img_shape[2], expand_animations=False)
    except tf.errors.InvalidArgumentError as e:
        tf.print(f"Warning: Could not decode image {path_tensor}. Error: {e}. Returning zeros.")
        return tf.zeros(img_shape, dtype=tf.float32)

    if len(img.shape) != 3 or img.shape[2] != img_shape[2]:
        tf.print(f"Warning: Image {path_tensor} has unexpected shape {img.shape}. Converting to RGB or returning zeros.")
        if img.shape[2] == 1: img = tf.image.grayscale_to_rgb(img)
        elif img.shape[2] == 4: img = img[:,:,:3]
        else: return tf.zeros(img_shape, dtype=tf.float32)

    img = tf.image.resize(img, [img_shape[0], img_shape[1]])
    img = tf.cast(img, tf.float32)

    if augment:
        img = augment_image(img)

    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    return img

def list_image_files_and_labels(directory):
    image_paths_by_label = {}
    label_to_name = {}
    name_to_label = {}
    current_label_id = 0
    class_names = sorted([d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))])

    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        if class_name not in name_to_label:
            name_to_label[class_name] = current_label_id
            label_to_name[current_label_id] = class_name
            image_paths_by_label[current_label_id] = []
            current_label_id += 1
        label_id = name_to_label[class_name]
        for fname in os.listdir(class_dir):
            if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths_by_label[label_id].append(os.path.join(class_dir, fname))
    all_unique_labels = sorted(list(label_to_name.keys()))
    return image_paths_by_label, label_to_name, name_to_label, all_unique_labels

def create_image_pairs(image_paths_by_label, all_unique_labels, num_pairs_per_anchor=1):
    pair_paths = []
    pair_labels = []
    if not all_unique_labels or not image_paths_by_label: return np.array(pair_paths), np.array(pair_labels)

    all_images_flat = [{'path': p, 'label': lbl} for lbl, paths in image_paths_by_label.items() for p in paths]
    if not all_images_flat: return np.array(pair_paths), np.array(pair_labels)

    random.shuffle(all_images_flat)

    for anchor_data in all_images_flat:
        anchor_path, anchor_label = anchor_data['path'], anchor_data['label']

        positive_candidates = [p for p in image_paths_by_label.get(anchor_label, []) if p != anchor_path]
        if positive_candidates:
            selected_positives = random.sample(positive_candidates, min(len(positive_candidates), num_pairs_per_anchor))
            for positive_path in selected_positives:
                pair_paths.append([anchor_path, positive_path])
                pair_labels.append(1.0)

        negative_label_choices = [l for l in all_unique_labels if l != anchor_label]
        if negative_label_choices:
            for _ in range(num_pairs_per_anchor):
                negative_label = random.choice(negative_label_choices)
                negative_candidates = image_paths_by_label.get(negative_label, [])
                if negative_candidates:
                    negative_path = random.choice(negative_candidates)
                    pair_paths.append([anchor_path, negative_path])
                    pair_labels.append(0.0)

    combined = list(zip(pair_paths, pair_labels))
    random.shuffle(combined)
    if combined:
        pair_paths, pair_labels = zip(*combined)
    else:
        return np.array([]), np.array([])
    return np.array(pair_paths), np.array(pair_labels)


def create_tf_dataset(pair_paths, pair_labels, batch_size, img_shape, augment_data=False):
    if len(pair_paths) == 0:
        print("Warning: pair_paths is empty. Cannot create tf.data.Dataset.")
        return None

    path1_list = [p[0] for p in pair_paths]
    path2_list = [p[1] for p in pair_paths]

    dataset = tf.data.Dataset.from_tensor_slices(((path1_list, path2_list), pair_labels))

    def _preprocess_pair(paths, label):
        path1, path2 = paths
        img1 = load_and_preprocess_image_tf_augmented(path1, img_shape, augment=augment_data)
        img2 = load_and_preprocess_image_tf_augmented(path2, img_shape, augment=augment_data)
        return (img1, img2), label

    dataset = dataset.shuffle(buffer_size=len(pair_labels))
    dataset = dataset.map(_preprocess_pair, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

print("\nPreparing training data (with augmentation)...")
train_images_by_label, _, _, train_unique_labels = list_image_files_and_labels(TRAIN_DIR)
train_pair_paths, train_pair_labels = create_image_pairs(train_images_by_label, train_unique_labels, num_pairs_per_anchor=3)
print(f"Number of training pairs for fine-tuning: {len(train_pair_paths)}")
train_dataset_finetune = create_tf_dataset(train_pair_paths, train_pair_labels, BATCH_SIZE, IMG_SHAPE, augment_data=True)

print("\nPreparing validation data (no augmentation)...")
test_images_by_label, _, _, test_unique_labels = list_image_files_and_labels(TEST_DIR)
val_pair_paths, val_pair_labels = create_image_pairs(test_images_by_label, test_unique_labels, num_pairs_per_anchor=3)
print(f"Number of validation pairs for fine-tuning: {len(val_pair_paths)}")
val_dataset_finetune = create_tf_dataset(val_pair_paths, val_pair_labels, BATCH_SIZE, IMG_SHAPE, augment_data=False)


# --- 5. Re-Compile and Train the Siamese Model (Fine-tuning) ---
if train_dataset_finetune and val_dataset_finetune:
    print("\nRe-compiling Siamese model for fine-tuning...")
    # CRITICAL: Re-compile the model to ensure changes in layer trainability take effect.
    siamese_model.compile(loss='binary_crossentropy',
                          optimizer=Adam(learning_rate=LEARNING_RATE_FINETUNE),
                          metrics=['accuracy'])

    print("\nFine-tuning Siamese model...")
    early_stopping_finetune = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=7,
        restore_best_weights=True,
        verbose=1
    )
    model_checkpoint_finetune = tf.keras.callbacks.ModelCheckpoint(
        FINETUNED_SIAMESE_MODEL_PATH,
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
    reduce_lr_finetune = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-7, # Adjusted to allow even lower LR if needed
        verbose=1
    )

    history_finetune = siamese_model.fit(
        train_dataset_finetune,
        epochs=EPOCHS_FINETUNE,
        validation_data=val_dataset_finetune,
        callbacks=[early_stopping_finetune, model_checkpoint_finetune, reduce_lr_finetune]
    )

    if history_finetune and history_finetune.history:
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history_finetune.history['loss'], label='Train Loss (Fine-tune)')
        plt.plot(history_finetune.history['val_loss'], label='Validation Loss (Fine-tune)')
        plt.title('Fine-tuning Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history_finetune.history['accuracy'], label='Train Accuracy (Fine-tune)')
        plt.plot(history_finetune.history['val_accuracy'], label='Validation Accuracy (Fine-tune)')
        plt.title('Fine-tuning Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.tight_layout()
        plt.savefig("/content/drive/MyDrive/ColabNotebooks/results/finetuning_v2_top_layers_history.png") # Changed name
        plt.show()

    print(f"Fine-tuned Siamese model (top layers) potentially saved to {FINETUNED_SIAMESE_MODEL_PATH} by ModelCheckpoint.")
    # Load the best model explicitly if ModelCheckpoint was used
    print("Loading the best model saved by ModelCheckpoint...")
    siamese_model = load_model(FINETUNED_SIAMESE_MODEL_PATH)


else:
    print("Fine-tuning training and/or validation dataset could not be created. Skipping fine-tuning.")


# --- 6. How to Use for New Landmark Identification (Conceptual) ---
def predict_similarity(img_path1, img_path2, model, img_shape):
    if model is None:
        print("Model not available for prediction.")
        return None
    img1_tensor = load_and_preprocess_image_tf_augmented(tf.constant(img_path1), img_shape, augment=False)
    img2_tensor = load_and_preprocess_image_tf_augmented(tf.constant(img_path2), img_shape, augment=False)

    if tf.reduce_sum(img1_tensor) == 0 or tf.reduce_sum(img2_tensor) == 0:
        print(f"Warning: One or both images ({os.path.basename(img_path1)}, {os.path.basename(img_path2)}) might not have loaded correctly for prediction.")

    img1_batch = tf.expand_dims(img1_tensor, axis=0)
    img2_batch = tf.expand_dims(img2_tensor, axis=0)

    try:
        prediction = model.predict([img1_batch, img2_batch], verbose=0)
        return prediction[0][0]
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None

print("\nLoading the BEST fine-tuned model for final predictions...")
try:
    # Ensure siamese_model holds the best version, especially if EarlyStopping restored weights
    # and ModelCheckpoint also saved. Loading explicitly from ModelCheckpoint path is safest.
    final_model_to_test = load_model(FINETUNED_SIAMESE_MODEL_PATH)
    print("Best fine-tuned model loaded for testing.")

    if val_pair_paths is not None and len(val_pair_paths) > 0 and final_model_to_test is not None:
        print("\nExample predictions on validation pairs (using the BEST fine-tuned model):")

        positive_indices = [i for i, label in enumerate(val_pair_labels) if label == 1.0]
        if positive_indices:
            idx = random.choice(positive_indices)
            path1, path2 = val_pair_paths[idx]
            label = val_pair_labels[idx]
            sim = predict_similarity(path1, path2, final_model_to_test, IMG_SHAPE)
            if sim is not None:
                print(f"Pair: ({os.path.basename(path1)}, {os.path.basename(path2)}), Actual Label: {label}, Predicted Similarity: {sim:.4f} (Same class)")
        else:
            print("No positive validation pairs found for example prediction.")

        negative_indices = [i for i, label in enumerate(val_pair_labels) if label == 0.0]
        if negative_indices:
            idx = random.choice(negative_indices)
            path1, path2 = val_pair_paths[idx]
            label = val_pair_labels[idx]
            sim = predict_similarity(path1, path2, final_model_to_test, IMG_SHAPE)
            if sim is not None:
                print(f"Pair: ({os.path.basename(path1)}, {os.path.basename(path2)}), Actual Label: {label}, Predicted Similarity: {sim:.4f} (Different classes)")
        else:
            print("No negative validation pairs found for example prediction.")
    else:
        print("\nSkipping example predictions as validation pairs or fine-tuned model is not available.")

except Exception as e:
    print(f"Error loading or using the fine-tuned model for final predictions: {e}")


print("\n--- Fine-tuning Script (Top Layers) Finished ---")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading the previously trained Siamese model...
Trained Siamese model loaded successfully.
Original Siamese model summary:


Model: "siamese_network_v2_corrected_lambda"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image_A       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_image_B       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feature_extractor_… │ (None, 1280)      │  2,257,984 │ input_image_A[0]… │
│ (Functional)        │                   │            │ input_image_B[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ L1_distance         │ (None, 1)         │          0 │ feature_extracto… │
│ (Lambda)            │                   │            │ feature_extracto… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ head_dense_1        │ (None, 128)       │        256 │ L1_distance[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ head_dropout_1      │ (None, 128)       │          0 │ head_dense_1[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ head_dense_2        │ (None, 64)        │      8,256 │ head_dropout_1[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ head_dropout_2      │ (None, 64)        │          0 │ head_dense_2[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ similarity_predict… │ (None, 1)         │         65 │ head_dropout_2[0… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,266,561 (8.65 MB)

 Trainable params: 8,577 (33.50 KB)

...

 Total params: 2,266,561 (8.65 MB)

 Trainable params: 418,177 (1.60 MB)

 Non-trainable params: 1,848,384 (7.05 MB)


Preparing training data (with augmentation)...
Number of training pairs for fine-tuning: 18300

Preparing validation data (no augmentation)...
Number of validation pairs for fine-tuning: 18300

Re-compiling Siamese model for fine-tuning...

Fine-tuning Siamese model...
Epoch 1/25


NameError: Exception encountered when calling Lambda.call().

[1mname 'K' is not defined[0m

Arguments received by Lambda.call():
  • inputs=['tf.Tensor(shape=(None, 1280), dtype=float32)', 'tf.Tensor(shape=(None, 1280), dtype=float32)']
  • mask=['None', 'None']
  • training=True

In [ ]:
# ... (plotting code from Phase 2 history) ...
SIAMESE_MODEL_PHASE2_PATH = '/content/drive/MyDrive/ColabNotebooks/results/siamese_mobilenetv2_landmark_model_phase2_toplayers.keras'
print(f"Phase 2 fine-tuned Siamese model saved to {SIAMESE_MODEL_PHASE2_PATH} (by ModelCheckpoint).")
print("This model was built with the corrected Lambda layer definition (with output_shape).")

# Load the best model saved by ModelCheckpoint for any immediate post-training use
print("Loading the best model from Phase 2 fine-tuning...")

# --- MODIFICATION ---
# Ensure unsafe deserialization is enabled before loading, as it contains a Python lambda.
# This should ideally be set once at the beginning of the script if loading multiple such models,
# but re-asserting it here or ensuring it's active is fine.
siamese_model_phase2_loaded = None # Initialize to ensure it exists
try:
    tf.keras.config.enable_unsafe_deserialization()
    print("Unsafe deserialization enabled (to load Python lambda in Lambda layer).")

    siamese_model_phase2_loaded = load_model(SIAMESE_MODEL_PHASE2_PATH) # Load into a new variable or overwrite 'siamese_model'
    print("Best Phase 2 model loaded successfully.")

except Exception as e:
    print(f"Error: Could not enable unsafe deserialization or load Phase 2 model: {e}")
    # siamese_model_phase2_loaded will remain None
# --- END MODIFICATION ---

# You can run predictions with this 'siamese_model_phase2_loaded' if needed.
# It's good practice to check if the model was loaded successfully.
if siamese_model_phase2_loaded is not None:
    if val_pair_paths is not None and len(val_pair_paths) > 0: # Assuming val_pair_paths is still in scope
        print("\nExample predictions on validation pairs (using the Phase 2 fine-tuned model):")
        # ... (rest of your prediction code using siamese_model_phase2_loaded) ...
        # For example, if your predict_similarity function and val_pair_paths are available:
        # positive_indices = [i for i, label in enumerate(val_pair_labels) if label == 1.0]
        # if positive_indices:
        #     idx = random.choice(positive_indices)
        #     path1, path2 = val_pair_paths[idx]
        #     label_val = val_pair_labels[idx]
        #     sim = predict_similarity(path1, path2, siamese_model_phase2_loaded, IMG_SHAPE)
        #     if sim is not None:
        #         print(f"Pair: ({os.path.basename(path1)}, {os.path.basename(path2)}), Actual Label: {label_val}, Predicted Similarity: {sim:.4f} (Same class)")
    else:
        print("\nValidation pairs not available for prediction with Phase 2 model.")
else:
    print("\nPhase 2 model was not loaded successfully. Skipping predictions.")

Phase 2 fine-tuned Siamese model saved to /content/drive/MyDrive/ColabNotebooks/results/siamese_mobilenetv2_landmark_model_phase2_toplayers.keras (by ModelCheckpoint).
This model was built with the corrected Lambda layer definition (with output_shape).
Loading the best model from Phase 2 fine-tuning...
Unsafe deserialization enabled (to load Python lambda in Lambda layer).
Best Phase 2 model loaded successfully.

Example predictions on validation pairs (using the Phase 2 fine-tuned model):
